# 时序数据 Dataset Interview：训练与部署安排（速查）
_更新时间：2026-01-29 04:30_

> 目标：在受限时间内，完成从数据理解 → 特征 → 训练 → 验证 → 产出物（模型/报告/可复现实验）→ 简易部署/交付 的一整套流程。  
> 口径：所有结论都以 **walk-forward/rolling** 的验证为准，避免泄露。


## 0. 环境与约束清单（开局 5 分钟）

- Python 版本、可用库（numpy/pandas/sklearn/matplotlib 等）
- 是否可联网（禁用 AI 工具）、是否可 pip 安装额外包
- 机器资源：CPU 核数、内存；是否有 GPU（通常无）
- 交付物：
  - 代码（notebook / .py）
  - 模型文件（joblib/pkl）
  - 结果表与图（png/csv）
  - presentation（PPT）

**固定约定（写在最上面避免忘）：**
- 随机种子：`SEED = 42`
- 时间列名：`time_col = ...`
- 目标列名：`target_col = ...`
- 分组列（如 asset_id / region / meter_id）：`group_col = ...`
- 评价指标：回归（MAE/RMSE/MAPE/Pinball），分类（AUC/F1），交易/策略（IC/Sharpe 等）


In [ ]:
import os
import json
import math
import time
import warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier

from sklearn.inspection import permutation_importance
import joblib

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
SEED = 42
np.random.seed(SEED)


## 1. 数据读取与最小数据字典（10–20 分钟）

### 1.1 读取
- 统一读入 dtype（尤其是时间、类别）
- 统一时区/格式
- 明确索引：`[group_col, time_col]` 或 `time_col`

### 1.2 最小数据字典（写在这里）
- 每列含义、单位、是否未来可得
- 是否存在目标泄露字段（例如未来统计量、结算价、label 编码等）
- 缺失机制：随机缺失 / 结构性缺失（停机、假日、断电）


In [ ]:
# 示例：根据现场数据调整
# df = pd.read_csv("train.csv")
# df[time_col] = pd.to_datetime(df[time_col])

# 统一排序（非常关键）
# if group_col in df.columns:
#     df = df.sort_values([group_col, time_col]).reset_index(drop=True)
# else:
#     df = df.sort_values([time_col]).reset_index(drop=True)

# df.head()


## 2. EDA（20–40 分钟，产出 3–6 张关键图）

### 2.1 时间范围与采样频率
- 起止时间
- 是否等间隔；是否需要 resample
- 每个 group 的覆盖率与缺口

### 2.2 目标分布与稳定性
- level/trend/seasonality
- 异常点与极端值
- 按时间切分的漂移（train vs later）

### 2.3 基础相关性（只做不泄露的）
- `corr(y, lagged features)`
- 按 group 的差异

**产出物**
- `fig_target_over_time.png`
- `fig_missing_heatmap_like.png`（用简单可视化替代热图也行）
- `eda_summary.csv`


In [ ]:
# 快速 EDA：缺失率
# miss = df.isna().mean().sort_values(ascending=False)
# miss.head(30)

# 目标随时间走势（无 group 时）
# import matplotlib.pyplot as plt
# plt.figure()
# df.set_index(time_col)[target_col].plot()
# plt.title("Target over time")
# plt.tight_layout()
# plt.savefig("fig_target_over_time.png", dpi=200)


## 3. 切分与验证（核心：防泄露）

### 3.1 规则
- 绝不 random split
- 采用 rolling / expanding window
- 若存在 group：
  - 先按时间切
  - 再在每个 fold 内按 group 聚合评估（避免某个大 group 主导）

### 3.2 常用配置
- 训练窗口：过去 N 天/周/月
- 验证窗口：未来 H（预测地平线）
- gap：避免用到 label 产生过程带来的近邻泄露（如对齐问题）

### 3.3 评估汇总
- 每 fold 指标
- 指标随时间曲线（稳定性）


In [ ]:
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def smape(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.maximum(eps, (np.abs(y_true) + np.abs(y_pred)) / 2.0)
    return np.mean(np.abs(y_pred - y_true) / denom)

# 仅 time_col 的情况
# tscv = TimeSeriesSplit(n_splits=5)

# 如果存在预测地平线 H，需要自定义切分（示意）
def rolling_splits(time_values: np.ndarray, n_splits=5, min_train_size=0.6):
    """返回 (train_idx, val_idx) 列表，按时间顺序滚动切分（无泄露）。
    time_values: 已按时间排序的一维数组（长度 n）。
    """
    n = len(time_values)
    start_train = int(n * min_train_size)
    step = (n - start_train) // n_splits
    splits = []
    for k in range(n_splits):
        train_end = start_train + k * step
        val_end = min(n, train_end + step)
        train_idx = np.arange(0, train_end)
        val_idx = np.arange(train_end, val_end)
        if len(val_idx) == 0:
            break
        splits.append((train_idx, val_idx))
    return splits


## 4. 特征工程（30–90 分钟，按优先级）

### 4.1 时间特征（无泄露）
- hour / dayofweek / month / holiday（若可用）
- sin/cos（周期特征）：日内、周内、年内

### 4.2 滞后与滚动统计（严格只用过去）
- lag: `y(t-1), y(t-2), ...`
- rolling: mean/std/min/max/quantile（窗口：3/7/14/28...）
- 对于 exogenous features：同样做 lag/rolling

### 4.3 分组特征（如资产/站点）
- group 的静态特征（地理/类别）
- group 的历史统计量（只用训练期累计）

### 4.4 目标变换
- log1p / Box-Cox（仅当目标为正且长尾）
- winsorize / clip（稳健性）

**约束**
- 任何“未来聚合”都属于泄露：例如用全数据均值做编码


In [ ]:
def add_time_features(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    out = df.copy()
    t = pd.to_datetime(out[time_col])
    out["hour"] = t.dt.hour
    out["dow"] = t.dt.dayofweek
    out["dom"] = t.dt.day
    out["month"] = t.dt.month

    # 周期特征：以一天为周期（hour）
    out["hour_sin"] = np.sin(2 * np.pi * out["hour"] / 24.0)
    out["hour_cos"] = np.cos(2 * np.pi * out["hour"] / 24.0)

    # 一周周期（dow）
    out["dow_sin"] = np.sin(2 * np.pi * out["dow"] / 7.0)
    out["dow_cos"] = np.cos(2 * np.pi * out["dow"] / 7.0)
    return out

def add_lag_rolling(df: pd.DataFrame, group_col: Optional[str], target_col: str,
                    lags=(1,2,3,7), rolls=(3,7,14)) -> pd.DataFrame:
    out = df.copy()
    if group_col and group_col in out.columns:
        g = out.groupby(group_col, sort=False)
        for L in lags:
            out[f"{target_col}_lag{L}"] = g[target_col].shift(L)
        for W in rolls:
            out[f"{target_col}_roll{W}_mean"] = g[target_col].shift(1).rolling(W).mean().reset_index(level=0, drop=True)
            out[f"{target_col}_roll{W}_std"]  = g[target_col].shift(1).rolling(W).std().reset_index(level=0, drop=True)
    else:
        for L in lags:
            out[f"{target_col}_lag{L}"] = out[target_col].shift(L)
        for W in rolls:
            out[f"{target_col}_roll{W}_mean"] = out[target_col].shift(1).rolling(W).mean()
            out[f"{target_col}_roll{W}_std"]  = out[target_col].shift(1).rolling(W).std()
    return out


## 5. 模型清单（按面试友好度排序）

### 5.1 Baseline（必须）
- Naive：`y_hat(t)=y(t-1)`（或季节 naive）
- Ridge/Lasso/ElasticNet（配合滞后/rolling/时间特征）

### 5.2 强基线（优先）
- HistGradientBoosting（速度快、效果强、对缺失更友好）
- RandomForest / GradientBoosting（数据不大时可用）

### 5.3 概率/区间预测（加分项）
- Quantile Regression（pinball loss）：
  - 用 `GradientBoostingRegressor(loss="quantile")` 做 p10/p50/p90
- 以覆盖率（coverage）评估

### 5.4 解释与稳健性（必须有一项）
- permutation importance
- 按时间段/按 group 的分层指标
- 残差随时间的结构（是否还剩季节性）


In [ ]:
def make_preprocess(df: pd.DataFrame, target_col: str):
    feature_cols = [c for c in df.columns if c != target_col]
    X = df[feature_cols]
    y = df[target_col]

    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X.columns if c not in num_cols]

    numeric = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False)),  # 稀疏安全
    ])

    categorical = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric, num_cols),
            ("cat", categorical, cat_cols),
        ],
        remainder="drop"
    )
    return pre, feature_cols, num_cols, cat_cols

def fit_eval_rolling(df_feat: pd.DataFrame, time_col: str, target_col: str,
                     group_col: Optional[str] = None, n_splits: int = 5):
    # 假设已按时间排序（若有 group：已按 group+time 排序）
    # 这里用全局顺序做示例；真实业务中可按“时间点集合”做切分
    time_values = pd.to_datetime(df_feat[time_col]).values
    splits = rolling_splits(time_values, n_splits=n_splits, min_train_size=0.6)

    metrics = []
    oof_pred = np.full(len(df_feat), np.nan)

    for k, (tr, va) in enumerate(splits, 1):
        train_df = df_feat.iloc[tr].copy()
        val_df   = df_feat.iloc[va].copy()

        pre, feature_cols, _, _ = make_preprocess(train_df.drop(columns=[time_col] if time_col in train_df.columns else []),
                                                  target_col=target_col)

        # 训练列：去掉 time_col（避免 onehot 爆炸）
        drop_cols = [time_col] if time_col in train_df.columns else []
        X_tr = train_df.drop(columns=[target_col] + drop_cols)
        y_tr = train_df[target_col]
        X_va = val_df.drop(columns=[target_col] + drop_cols)
        y_va = val_df[target_col]

        model = HistGradientBoostingRegressor(random_state=SEED)
        pipe = Pipeline(steps=[("pre", pre), ("model", model)])
        pipe.fit(X_tr, y_tr)
        pred = pipe.predict(X_va)

        oof_pred[va] = pred

        fold = {
            "fold": k,
            "n_train": len(tr),
            "n_val": len(va),
            "MAE": float(mean_absolute_error(y_va, pred)),
            "RMSE": float(rmse(y_va, pred)),
            "SMAPE": float(smape(y_va, pred)),
        }
        metrics.append(fold)

    return pd.DataFrame(metrics), oof_pred


## 6. 调参安排（时间受限版本）

### 6.1 原则
- 先做 feature，再做模型
- 只调 3–6 个关键参数
- 用同一套 rolling split 做对比
- 记录每次实验的配置与指标（可复现）

### 6.2 经验参数范围（以 HGB 为例）
- `max_depth`: 3–8
- `learning_rate`: 0.03–0.2
- `max_leaf_nodes`: 15–127
- `min_samples_leaf`: 20–200
- `l2_regularization`: 0–1

### 6.3 早停与算力控制
- 数据过大：先抽样/降频做方向验证
- 再全量跑最终模型


In [ ]:
from sklearn.model_selection import ParameterGrid

def grid_search_small(df_feat: pd.DataFrame, time_col: str, target_col: str, param_grid: Dict, n_splits: int = 5):
    results = []
    for params in ParameterGrid(param_grid):
        # 自定义 fit_eval，简化：直接在函数里替换模型参数
        time_values = pd.to_datetime(df_feat[time_col]).values
        splits = rolling_splits(time_values, n_splits=n_splits, min_train_size=0.6)

        fold_scores = []
        for tr, va in splits:
            train_df = df_feat.iloc[tr].copy()
            val_df   = df_feat.iloc[va].copy()

            drop_cols = [time_col] if time_col in train_df.columns else []
            pre, _, _, _ = make_preprocess(train_df.drop(columns=drop_cols), target_col=target_col)

            X_tr = train_df.drop(columns=[target_col] + drop_cols)
            y_tr = train_df[target_col]
            X_va = val_df.drop(columns=[target_col] + drop_cols)
            y_va = val_df[target_col]

            model = HistGradientBoostingRegressor(random_state=SEED, **params)
            pipe = Pipeline(steps=[("pre", pre), ("model", model)])
            pipe.fit(X_tr, y_tr)
            pred = pipe.predict(X_va)
            fold_scores.append(mean_absolute_error(y_va, pred))

        results.append({**params, "MAE_mean": float(np.mean(fold_scores)), "MAE_std": float(np.std(fold_scores))})
    return pd.DataFrame(results).sort_values("MAE_mean")


## 7. 训练产出物（必须落盘）

- `model.joblib`：训练好的 pipeline（含预处理）
- `metrics.csv`：fold 指标与汇总
- `oof_predictions.csv`：时间戳 + 预测 + 真实值（便于画图与诊断）
- `feature_importance.csv`：permutation importance 或模型自带重要性
- `run_config.json`：本次训练配置（窗口、特征、参数、指标）


In [ ]:
def save_run_artifacts(output_dir: str, pipe, metrics_df: pd.DataFrame, df_feat: pd.DataFrame,
                       time_col: str, target_col: str, oof_pred: np.ndarray, config: Dict):
    os.makedirs(output_dir, exist_ok=True)

    joblib.dump(pipe, os.path.join(output_dir, "model.joblib"))
    metrics_df.to_csv(os.path.join(output_dir, "metrics.csv"), index=False)

    oof = df_feat[[time_col]].copy()
    oof["y_true"] = df_feat[target_col].values
    oof["y_pred"] = oof_pred
    oof.to_csv(os.path.join(output_dir, "oof_predictions.csv"), index=False)

    with open(os.path.join(output_dir, "run_config.json"), "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)

    return output_dir


## 8. 部署/交付安排（面试版：最小可运行）

### 8.1 批量打分（batch scoring）
- 输入：`new_data.csv`
- 输出：`predictions.csv`（含 time/group/key）
- 约束：严格复用训练期同一套特征逻辑与 pipeline

### 8.2 在线/准在线（streaming 的表达方式）
- 本质：每来一个新时间点，更新 lag/rolling 缓存，调用 `model.predict`
- 需要一个状态对象：保存最近 W 个观测（per group）
- 面试中只需说明接口与状态，不需要实现完整系统

### 8.3 监控（最小 3 项）
- 数据漂移：特征均值/方差、缺失率
- 预测漂移：残差分布、分位数变化
- 性能回归：按周/月 rolling 评估指标

### 8.4 复训节奏（写成可执行计划）
- 每天/每周固定时间增量训练
- 触发式复训：漂移超过阈值、指标显著下降


In [ ]:
def batch_predict(model_path: str, raw_path: str, out_path: str,
                  time_col: str, group_col: Optional[str] = None) -> str:
    pipe = joblib.load(model_path)
    df_new = pd.read_csv(raw_path)
    df_new[time_col] = pd.to_datetime(df_new[time_col])

    # 排序
    if group_col and group_col in df_new.columns:
        df_new = df_new.sort_values([group_col, time_col]).reset_index(drop=True)
    else:
        df_new = df_new.sort_values([time_col]).reset_index(drop=True)

    # 这里需要复用训练时的特征工程逻辑：add_time_features + add_lag_rolling 等
    # 现场做法：把特征工程封装成函数并在训练/预测统一调用
    # df_new_feat = add_time_features(df_new, time_col)
    # df_new_feat = add_lag_rolling(df_new_feat, group_col, target_col=..., ...)

    # 假设 df_new_feat 已准备好，并且剔除了目标列
    # pred = pipe.predict(df_new_feat.drop(columns=[time_col]))
    # out = df_new[[time_col] + ([group_col] if group_col else [])].copy()
    # out["y_pred"] = pred
    # out.to_csv(out_path, index=False)

    return out_path


## 9. Presentation（10–20 分钟出稿结构）

### 9.1 1 页：问题与数据
- 目标定义、预测 horizon、评估指标
- 数据规模、时间跨度、分组数量

### 9.2 1 页：EDA 关键发现
- trend/seasonality/缺失/异常
- 漂移：早期 vs 后期

### 9.3 1 页：方法与验证
- 特征：时间特征 + lag/rolling +（可选）Fourier
- 验证：rolling split + 指标曲线

### 9.4 1 页：结果与解释
- baseline vs 最终模型
- 重要特征
- 失败案例（误差最大的时段/组）

### 9.5 1 页：落地与后续
- batch scoring 接口
- 监控与复训节奏
- 可能改进：外生变量、分层模型、区间预测


## 10. 最终检查清单（提交前 3 分钟）

- [ ] 时间排序正确
- [ ] 不存在未来信息（含统计量/编码/对齐）
- [ ] rolling split 指标稳定、无异常飙升（疑似泄露）
- [ ] 保存了 model + metrics + oof + config
- [ ] 图表可复现，路径相对、命名清晰
- [ ] PPT 结论与 notebook 输出一致
